# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. <br/>
**Задача:** <br/>   **разобраться влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок.**<br/>   Входные данные от банка — статистика о платёжеспособности клиентов. <br/>
**Вопросы:** <br/> 
Зависимость возврата кредита в срок от:<br/>
**наличия детей  
семейного положения  
уровня дохода  
целей кредита**

Результаты исследования будут 
учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.
### Описание данных
**children** — количество детей в семье  
**days_employed** — общий трудовой стаж в днях  
**dob_years** — возраст клиента в годах  
**education** — уровень образования клиента  
**education_id** — идентификатор уровня образования  
**family_status** — семейное положение  
**family_status_id** — идентификатор семейного положения  
**gender** — пол клиента  
**income_type** — тип занятости  
**debt** — имел ли задолженность по возврату кредитов  
**total_income** — ежемесячный доход  
**purpose** — цель получения кредита  

### Шаг 1. Прочитаем данные, импортируем библиотеки, изучим общую информацию. 

In [3]:
import pandas as pd
#from pymystem3 import Mystem
#from nltk.stem import SnowballStemmer
#from collections import Counter
import numpy as np
import math as mth

In [4]:
debtors = pd.read_csv('/data.csv')
debtors.head(20) 

In [ ]:
debtors.tail(20)

In [ ]:
debtors.info()

In [ ]:
debtors.describe()

In [ ]:
debtors.duplicated().sum()

In [ ]:
#найдем пропуски
debtors.isnull().sum()

In [ ]:
debtors[debtors['days_employed'].isin(['NaN'])]

In [ ]:
# смотрим значений по детям?
debtors.sort_values('children')['children'].value_counts().sort_index()

In [ ]:
debtors['children'].value_counts()

In [ ]:
debtors['gender'].value_counts()

In [ ]:
debtors['family_status'].unique()

In [ ]:
debtors['purpose'].unique()

In [ ]:
debtors['purpose'].unique()

In [ ]:
debtors['income_type'].unique()

In [ ]:
#debtors['dob_years'].unique()
debtors['dob_years'][(debtors['dob_years'] < 19) & (debtors['dob_years'] > 0)].count()
debtors['dob_years'][debtors['dob_years'] == 0].count()

### Выводы по данным

1. **children**: -1 ребенок (47 значений) - артефакт? можно удалить, заменить на 0 или зачесть в +1. Заменю на 1 - поскольку прилипший ноль в этих данных похоже норма, а количество значений не критично для анализа. 20 детей (76 значений) - многоват. конечно, но это для меня, а так - многодетная семья.
2. **days_employed**: **ПРОПУСКИ**: трудно предположить откуда, информацию о заемщике обычно собирают тщательно, может потерялись при выгрузке. В поставленных в ТЗ задачах не фигурирует и, можно предположить, что влияние на надежность заемщика данный параметр если и оказывает, то косвенноое.Пропуски можно заменить нулями или взять среднее значение по группе занятости. Средее не нашел как  Большое количество **значений с минусом** - можем только предполагать, что минус нарисовался программно или при неправильном заполнении. Будем считат, что это артефакт и **возьмем по модулю**, сообщим о проблеме, если есть кому. есть **выбивающиеся значения**. Можно **поменять тип данных** для красоты и удобства.
3. **dob_years**: нулевые значения в возрасте (101). несовершеннолетних нет. 
3. **education**, **family_status**: регистр скачет
4. **total_income**: ПРОПУСКИ в тех же строках, что и стаж, но связи не вижу, заполнить можно по типу занятости. Интересно сколько пенсионеры получают?! тип данных не нравится. вообще ежемесячный доход важный параметр  - надо посмотреть повнимательней. 
5. **purpose**: цели дублируются - нужно категорировать

Есть дубли - 54 шт.